In [1]:
import os
os.environ['OPENAI_API_KEY'] = "EXAMPLE"

In [14]:
cd /content/drive/MyDrive/Colab Notebooks/langchainsawman/week4

/content/drive/MyDrive/Colab Notebooks/langchainsawman/week4


# 텍스트 임베딩 모델
##### 임베딩은 텍스트 조각의 벡터 표현을 만든다.
##### 이는 벡터 공간에서 텍스트에 대해 생각할 수 있고, 벡터 공간에서 가장 유사한 텍스트 조각을 찾는 의미 검색과 같은 작업을 수핼할 수 있다는 의미이다.
##### Langchain의 기본 Embedding 클래스는 두 가지 메서드를 제공한다.
##### 하나는 문서로를 포함하는 방법이고, 다른 하나는 쿼리를 포함하는 방법이다.
##### 전자는 여러 텍스트를 입력으로 받고, 후자는 단일 텍스트를 입력으로 받는다.

In [ ]:
!pip install langchain-openai

In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [4]:
# 텍스트 문서 임베딩
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [5]:
# 단일 쿼리 임베딩
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.005384807424727807,
 -0.0005522561790177147,
 0.03896066510130955,
 -0.002939867294003909,
 -0.008987877434176603]

## 캐싱
##### 임베딩을 다시 계산할 필요가 없도록 저장하거나 임시로 캐시할 수 있다.
##### 캐싱 임베딩은 CacheBackedEmbeddings이고, 텍스트는 해시되고 해시는 캐시의 키로 사용된다.
##### CacheBackedEmbeddings의 매개변수는 아래와 같다.



*   embedder : 임베딩에 사용할 임베더
*   document_embedding_cache : 문서 임베딩을 캐싱하기 위한 ByteStore
*   batch_size : 스토어 업데이트 사이에 포함할 문서 수(기본 값 None, optional)
*   namespace : 문서 캐시에 사용할 네임스페이스(기본 값 "", optional)



In [ ]:
!pip install -qU langchain

In [9]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.2 MB/s eta 0:00:00


In [8]:
from langchain.embeddings import CacheBackedEmbeddings

### 벡터 스토어
##### 임베딩 저장을 위해 로컬 파일 시스템을 사용하고, 검색을 위해 FAISS 벡터 저장소를 사용하자

In [16]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

underlying_embeddings = OpenAIEmbeddings()

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

In [17]:
list(store.yield_keys())

[]

In [18]:
raw_documents = TextLoader("whatsapp_chat.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [19]:
%%time
db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 54.8 ms, sys: 3.94 ms, total: 58.7 ms
Wall time: 233 ms


In [20]:
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 2.45 ms, sys: 978 µs, total: 3.43 ms
Wall time: 6.88 ms


##### ByteStore를 교환할려면 CacheBackedEmbeddings을 만들 때, 비영구 객체를 만들어라(InMemoryByteStore)

In [21]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)